In [12]:
import pandas as pd
import csv
import requests
from bs4 import BeautifulSoup
import requests
import pymongo
import datetime
import json
import requests
import time

# Google developer API key
from config import api_key

## Scrape Data

In [13]:
url = 'https://www.worldometers.info/coronavirus/'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_html(r.text)

df = dfs[0]

df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop
0,World,1491465,"+60,546",87444.0,"+5,410",319060.0,1084961,48015.0,191.0,11.2,NaN,NaN
1,USA,418410,"+18,075",14240.0,"+1,399",22184.0,381986,9224.0,1264.0,43.0,2172976.0,6565.0
2,Spain,146690,"+4,748",14673.0,+628,48021.0,83996,7069.0,3137.0,314.0,355000.0,7593.0
3,Italy,139422,"+3,836",17669.0,+542,26491.0,95262,3693.0,2306.0,292.0,807125.0,13349.0
4,France,112950,"+3,881",10869.0,+541,21254.0,80827,7148.0,1730.0,167.0,224254.0,3436.0
...,...,...,...,...,...,...,...,...,...,...,...,...
208,South Sudan,2,NaN,NaN,NaN,NaN,2,NaN,0.2,NaN,NaN,NaN
209,Saint Pierre Miquelon,1,NaN,NaN,NaN,NaN,1,NaN,173.0,NaN,NaN,NaN
210,Timor-Leste,1,NaN,NaN,NaN,NaN,1,NaN,0.8,NaN,NaN,NaN
211,China,81802,+62,3333.0,+2,77279.0,1190,189.0,57.0,2.0,NaN,NaN


## Check for matches in database

In [29]:
countriesScraped = list(df['Country,Other'])
# countries
# The default port used by MongoDB is 27017
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.corona_db
collection = db.corona_data
results = db.corona_data.find()[0]
existingCountries = list(results['Country,Other'].values())

# for country in countriesScraped:
#     if country not in existingCountries:
#         print(country)
#         Run api call to get coords


countries = list(df['Country,Other'])
countryCount = 0
coordinates = []
for country in countries:
#     print(country)
    url = (f'https://maps.googleapis.com/maps/api/geocode/json?address={country}&key={api_key}')
    # fix this with not in []
    if country not in ['World','Total:']:
        print(country)
        try:
            response = requests.get(url).json()
            result = response['results'][0]['geometry']['location']
            loc = [result['lat'],result['lng']]

            print(loc)
        except:
            print('Failed Request')
            loc = [0,0]
        coordinates.append(loc)
        time.sleep(3)
    else:
        loc = [0,0]
        coordinates.append(loc)
    
    
    countryCount +=1 


USA
[37.09024, -95.712891]
Spain
[40.46366700000001, -3.74922]
Italy
[41.87194, 12.56738]
France
[46.227638, 2.213749]
Germany
[51.165691, 10.451526]
Iran
[32.427908, 53.688046]
UK
[55.378051, -3.435973]
Turkey
[38.963745, 35.243322]
Belgium
[50.503887, 4.469936]
Switzerland
[46.818188, 8.227511999999999]
Netherlands
[52.132633, 5.291265999999999]
Canada
[56.130366, -106.346771]
Brazil
[-14.235004, -51.92528]
Portugal
[39.39987199999999, -8.224454]
Austria
[47.516231, 14.550072]
S. Korea
[35.907757, 127.766922]
Israel
[31.046051, 34.851612]
Russia
[61.52401, 105.318756]
Sweden
[60.12816100000001, 18.643501]
Norway
[60.47202399999999, 8.468945999999999]
Ireland
[53.1423672, -7.692053599999999]
Australia
[-25.274398, 133.775136]
India
[20.593684, 78.96288]
Chile
[-35.675147, -71.542969]
Denmark
[56.26392, 9.501785]
Czechia
[49.81749199999999, 15.472962]
Poland
[51.919438, 19.145136]
Romania
[45.943161, 24.96676]
Japan
[36.204824, 138.252924]
Pakistan
[30.375321, 69.34511599999999]
Malays

In [33]:
df.to_json('countries2.json')

In [34]:
# Insert document into collection
with open('countries2.json') as f:
    countries_data = json.load(f)



In [35]:
countries_data

{'Country,Other': {'0': 'World',
  '1': 'USA',
  '2': 'Spain',
  '3': 'Italy',
  '4': 'France',
  '5': 'Germany',
  '6': 'Iran',
  '7': 'UK',
  '8': 'Turkey',
  '9': 'Belgium',
  '10': 'Switzerland',
  '11': 'Netherlands',
  '12': 'Canada',
  '13': 'Brazil',
  '14': 'Portugal',
  '15': 'Austria',
  '16': 'S. Korea',
  '17': 'Israel',
  '18': 'Russia',
  '19': 'Sweden',
  '20': 'Norway',
  '21': 'Ireland',
  '22': 'Australia',
  '23': 'India',
  '24': 'Chile',
  '25': 'Denmark',
  '26': 'Czechia',
  '27': 'Poland',
  '28': 'Romania',
  '29': 'Japan',
  '30': 'Pakistan',
  '31': 'Malaysia',
  '32': 'Ecuador',
  '33': 'Philippines',
  '34': 'Saudi Arabia',
  '35': 'Luxembourg',
  '36': 'Indonesia',
  '37': 'Peru',
  '38': 'Mexico',
  '39': 'Serbia',
  '40': 'UAE',
  '41': 'Finland',
  '42': 'Thailand',
  '43': 'Panama',
  '44': 'Qatar',
  '45': 'Dominican Republic',
  '46': 'Greece',
  '47': 'Colombia',
  '48': 'South Africa',
  '49': 'Argentina',
  '50': 'Ukraine',
  '51': 'Singapore',
 

In [36]:
collection.insert_one(countries_data)